In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option("display.max_columns",100)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, auc, confusion_matrix

from scipy.stats import kurtosis, skew

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import os
print(os.listdir("../input"))

import gc

# Any results you write to the current directory are saved as output.

In [55]:
df_train = pd.read_csv('../input/application_train.csv')
df_train.head()

In [56]:
df_test = pd.read_csv('../input/application_test.csv')
#df_test.head()

# merge (and split) train and test together

I will merge train and test together for easy preprocessing and I will demonstrate how to split them again for when I am done.

In [57]:
print(df_train.shape)
#print(df_test.shape)

In [58]:
y = df_train['TARGET']
X = df_train.drop(['TARGET'], axis=1)
X.head()

In [59]:
print(X.shape)
X = pd.concat([X, df_test], axis=0)
X.shape

In [33]:
#X_split = X[:307511]
#X_split.shape

In [34]:
#X_split.head()

# split features of X in categorical and non-categorical

I have the following in mind: I want to use MICE imputer for all the non categorical features (floats, ints etc.) and use pd.get_dummies for all the categorical feats (the object types)

In [60]:
# Categroical features for X
cat_X = X.drop(['SK_ID_CURR'], axis=1)
cat_X = [col for col in X.columns if X[col].dtype == 'object']
SK_ID = X['SK_ID_CURR']

cat_X = X[cat_X]
cat_X.head()

In [61]:
# Non categorical features for X

ncat_X = [col for col in X.columns if X[col].dtype != 'object']
ncat_X = X[ncat_X]
cols_ncat_X = ncat_X.columns.values
ncat_X.head()

In [62]:
# MICE imputer for numerical feats
#from sklearn.impute import SimpleImputer, MICEImputer
#mice = MICEImputer(initial_strategy='median', n_imputations=5, n_nearest_features=5, verbose=True)
#ncat_X = mice.fit_transform(ncat_X)
ncat_X = ncat_X.fillna(0)


In [63]:
ncat_X = pd.DataFrame(ncat_X, columns=cols_ncat_X)
ncat_X['SK_ID_CURR'] = SK_ID.values

ncat_X.head(10)

In [64]:
ncat_X['PERC_INCOME'] = ncat_X.AMT_CREDIT/ncat_X.AMT_INCOME_TOTAL
ncat_X.head()

In [65]:
ncat_X['GOODS_BIGGER_CREDIT'] = np.where(ncat_X.AMT_GOODS_PRICE > ncat_X.AMT_CREDIT, 1, 0)
ncat_X['AMOUNT_NOT_CREDIT'] = ncat_X.AMT_CREDIT - ncat_X.AMT_GOODS_PRICE
ncat_X.head(20)

In [66]:
print( 'untouched kurtosis of normal distribution (should be 0): {}'.format( kurtosis(ncat_X.AMT_INCOME_TOTAL) ))
print( 'untouched skewness of normal distribution (should be 0): {}'.format( skew(ncat_X.AMT_INCOME_TOTAL) ))

print( 'Log kurtosis of normal distribution (should be 0): {}'.format( kurtosis(np.log(ncat_X.AMT_INCOME_TOTAL)) ))
print( 'Log skewness of normal distribution (should be 0): {}'.format( skew(np.log(ncat_X.AMT_INCOME_TOTAL)) ))


f, (ax1, ax2) = plt.subplots(2)
sns.kdeplot(ncat_X.AMT_INCOME_TOTAL, ax=ax1)
sns.kdeplot(np.log(ncat_X.AMT_INCOME_TOTAL), ax=ax2)
ax1.set_title('Normal AMT_INCOME_TOTAL')
ax2.set_title('Log AMT_INCOME_TOTAL');


In [67]:
ncat_X['AMT_INCOME_TOTAL'] = np.log(ncat_X.AMT_INCOME_TOTAL)

In [68]:
print( 'untouched kurtosis of normal distribution (should be 0): {}'.format( kurtosis(ncat_X.AMT_CREDIT) ))
print( 'untouched skewness of normal distribution (should be 0): {}'.format( skew(ncat_X.AMT_CREDIT) ))

print( 'Log kurtosis of normal distribution (should be 0): {}'.format( kurtosis(np.log(ncat_X.AMT_CREDIT)) ))
print( 'Log skewness of normal distribution (should be 0): {}'.format( skew(np.log(ncat_X.AMT_CREDIT)) ))

f, (ax1, ax2) = plt.subplots(2)
sns.kdeplot(ncat_X.AMT_CREDIT, ax=ax1)
sns.kdeplot(np.log(ncat_X.AMT_CREDIT), ax=ax2)
ax1.set_title('Normal AMT_CREDIT')
ax2.set_title('Log AMT_CREDIT');

In [69]:
ncat_X['AMT_CREDIT'] = np.log(ncat_X.AMT_CREDIT)

In [70]:
print( 'untouched kurtosis of normal distribution (should be 0): {}'.format( kurtosis(ncat_X.AMT_ANNUITY) ))
print( 'untouched skewness of normal distribution (should be 0): {}'.format( skew(ncat_X.AMT_ANNUITY) ))

print( 'Log kurtosis of normal distribution (should be 0): {}'.format( kurtosis(np.log(ncat_X.AMT_ANNUITY)) ))
print( 'Log skewness of normal distribution (should be 0): {}'.format( skew(np.log(ncat_X.AMT_ANNUITY)) ))

f, (ax1, ax2) = plt.subplots(2)
sns.kdeplot(ncat_X.AMT_ANNUITY, ax=ax1)
sns.kdeplot(np.log(ncat_X.AMT_ANNUITY), ax=ax2)
ax1.set_title('Normal AMT_ANNUITY')
ax2.set_title('Log AMT_ANNUITY');

In [71]:
ncat_X['AMT_ANNUITY'] = np.log(ncat_X.AMT_ANNUITY)

In [72]:
print( 'untouched kurtosis of normal distribution (should be 0): {}'.format( kurtosis(ncat_X.AMT_GOODS_PRICE) ))
print( 'untouched skewness of normal distribution (should be 0): {}'.format( skew(ncat_X.AMT_GOODS_PRICE) ))

print( 'Log kurtosis of normal distribution (should be 0): {}'.format( kurtosis(np.log(ncat_X.AMT_GOODS_PRICE)) ))
print( 'Log skewness of normal distribution (should be 0): {}'.format( skew(np.log(ncat_X.AMT_GOODS_PRICE)) ))

f, (ax1, ax2) = plt.subplots(2)
sns.kdeplot(ncat_X.AMT_GOODS_PRICE, ax=ax1)
sns.kdeplot(np.log(ncat_X.AMT_GOODS_PRICE), ax=ax2)
ax1.set_title('Normal AMT_GOODS_PRICE')
ax2.set_title('Log AMT_GOODS_PRICE');

In [73]:
ncat_X['AMT_GOODS_PRICE'] = np.log(ncat_X.AMT_GOODS_PRICE)

In [74]:
print( 'untouched kurtosis of normal distribution (should be 0): {}'.format( kurtosis(ncat_X.DAYS_EMPLOYED) ))
print( 'untouched skewness of normal distribution (should be 0): {}'.format( skew(ncat_X.DAYS_EMPLOYED) ))

print( 'Log kurtosis of normal distribution (should be 0): {}'.format( kurtosis(np.log(ncat_X.DAYS_EMPLOYED)) ))
print( 'Log skewness of normal distribution (should be 0): {}'.format( skew(np.log(ncat_X.DAYS_EMPLOYED)) ))

f, (ax1, ax2) = plt.subplots(2)
sns.kdeplot(ncat_X.DAYS_EMPLOYED, ax=ax1)
sns.kdeplot(np.log(ncat_X.DAYS_EMPLOYED), ax=ax2)
ax1.set_title('Normal AMT_GOODDAYS_EMPLOYEDS_PRICE')
ax2.set_title('Log DAYS_EMPLOYED');

In [75]:
ncat_X.head()

In [76]:
ncat_X['DAYS_BIRTH'] = ncat_X.DAYS_BIRTH.apply(lambda x: x/-365)
ncat_X['DAYS_EMPLOYED'] = ncat_X.DAYS_EMPLOYED.apply(lambda x: x/-365)
ncat_X['DAYS_REGISTRATION'] = ncat_X.DAYS_REGISTRATION.apply(lambda x: x/-365)
ncat_X['DAYS_ID_PUBLISH'] = ncat_X.DAYS_ID_PUBLISH.apply(lambda x: x/-365)
ncat_X['DAYS_LAST_PHONE_CHANGE'] = ncat_X.DAYS_LAST_PHONE_CHANGE.apply(lambda x: x/-365)
ncat_X.head()

In [77]:
ncat_X.shape

In [83]:
cat_X.head()

In [86]:
'''# pd dummies for categorical feats
cat_X = pd.get_dummies(cat_X)
cat_X = pd.concat([SK_ID, cat_X], axis=1)

cat_X.head(10)'''

for x in cat_X.columns.values:
    print(x)
    keys = cat_X[x].unique()
    dicts = dict(zip(keys, range(len(keys))))
    cat_X[x] = cat_X[x].map(dicts).astype(int)

cat_X.head()

In [89]:
cat_X = pd.concat([SK_ID, cat_X], axis=1)

In [90]:
cat_X.shape

In [91]:
X = ncat_X.merge(cat_X, how='left', on='SK_ID_CURR')

In [92]:
X.head()

In [93]:
X.shape

In [94]:
gc.enable()
del ncat_X
del cat_X
gc.collect()

Basically I will now do the same thing for the other data files.

In [95]:
credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
credit_card_balance.head()

In [96]:
credit_card_balance['NAME_CONTRACT_STATUS'] = pd.get_dummies(credit_card_balance['NAME_CONTRACT_STATUS'])
credit_card_balance = credit_card_balance.fillna(0)
credit_card_balance.head(10)

In [97]:
ID_PREV = credit_card_balance[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
credit_card_balance['SK_ID_PREV'] = credit_card_balance['SK_ID_CURR'].map(ID_PREV['SK_ID_PREV'])

In [98]:
credit_card_mean = credit_card_balance.groupby('SK_ID_CURR').mean()
credit_card_mean.columns = ['cc_' + col for col in credit_card_mean.columns]
credit_card_mean = credit_card_mean.reset_index()
credit_card_mean.head()

In [99]:
X = X.merge(right=credit_card_mean, how='left', on='SK_ID_CURR')
X.head(10)

In [100]:
gc.enable()
del credit_card_balance
del credit_card_mean
gc.collect()

In [101]:
X.shape

In [102]:
POS_CASH_balance = pd.read_csv('../input/POS_CASH_balance.csv') 
POS_CASH_balance.head()

In [103]:
POS_CASH_balance['NAME_CONTRACT_STATUS'] = pd.get_dummies(POS_CASH_balance['NAME_CONTRACT_STATUS'])
POS_CASH_balance = POS_CASH_balance.fillna(0)

In [104]:
pcb_count = POS_CASH_balance[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
POS_CASH_balance['SK_ID_PREV'] = POS_CASH_balance['SK_ID_CURR'].map(pcb_count['SK_ID_PREV'])

In [105]:
pos_cash_mean = POS_CASH_balance.groupby('SK_ID_CURR').mean()

pos_cash_mean.columns = ['pc_' + col for col in pos_cash_mean.columns]
pos_cash_mean = pos_cash_mean.reset_index()
pos_cash_mean.head()

In [106]:
X = X.merge(right=pos_cash_mean, how='left', on='SK_ID_CURR')
X.shape

In [107]:
gc.enable()
del POS_CASH_balance
del pos_cash_mean
gc.collect()

In [108]:
bureau = pd.read_csv('../input/bureau.csv')
bureau.head()

In [109]:
bureau_mean = bureau.groupby('SK_ID_CURR').mean()
bureau_mean['buro_count'] = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']
bureau_mean.columns = ['b_' + col for col in bureau_mean.columns]
X = X.merge(right=bureau_mean.reset_index(), how='left', on='SK_ID_CURR')
X.shape

In [110]:
gc.enable()
del bureau
del bureau_mean
gc.collect()


In [111]:
previous_application = pd.read_csv('../input/previous_application.csv')
previous_application.head()

In [112]:
previous_app_mean = previous_application.groupby('SK_ID_CURR').mean()
previous_app_mean['SK_ID_PREV'] = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()['SK_ID_PREV']
previous_app_mean.columns = ['pa_' + col for col in previous_app_mean.columns]
previous_app_mean.head()

In [113]:
X = X.merge(right=previous_app_mean.reset_index(), how='left', on='SK_ID_CURR')
X.shape

In [114]:
gc.enable()
del previous_app_mean
del previous_application
gc.collect()

In [115]:
installments_payments = pd.read_csv('../input/installments_payments.csv')
installments_payments.head()

In [116]:
install_pay_mean = installments_payments.groupby('SK_ID_CURR').mean()
install_pay_mean['SK_ID_PREV'] = installments_payments[['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').count()['SK_ID_PREV']
install_pay_mean.columns = ['ip_' + col for col in install_pay_mean.columns]
X = X.merge(right=install_pay_mean.reset_index(), how='left', on='SK_ID_CURR')

In [117]:
gc.enable()
del installments_payments
del install_pay_mean
gc.collect()

In [118]:
X.head()

In [119]:
columns_X = X.columns
X = X.fillna(0)
df_train = X[:307511]
df_test = X[307511:]


In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=123)

In [121]:
import lightgbm as lgb
lgb_train = lgb.Dataset(data=X_train, label=y_train)
lgb_test = lgb.Dataset(data=X_test, label=y_test)

In [122]:
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 0 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1}
model = lgb.train(params, lgb_train, valid_sets=lgb_test, early_stopping_rounds=150, verbose_eval=200)

In [123]:
lgb.plot_importance(model, figsize=(12, 25), max_num_features=100);

In [124]:
lgb_preds = model.predict(df_test)

In [125]:
df_test['TARGET'] = lgb_preds

In [126]:
lgb_pred = df_test[['SK_ID_CURR', 'TARGET']].to_csv('LGB_prediction2.csv', index=False)